# Daten via WFS beziehen

Ein WFS-Dienst liefert Vektor-Geodaten. Überlicherweise sind bei einem WFS mehrere Layer enthalten, welche jeweils unterschiedliche Daten enthalten können. [Esri hat eine gute Anleitung](https://enterprise.arcgis.com/de/server/10.3/publish-services/linux/communicating-with-a-wfs-service-in-a-web-browser.htm), wie mit einem WFS-Server kommuniziert werden kann.

In diesem Notebook wird gezeigt, wie Daten via WFS abgefragt werden können (z.B. als GeoJSON) um diese dann weiterzuverarbeiten. Die Daten können dabei auf einer Karte dargestellt werden, oder auch als tabellarische Daten aufbereitet werden.
Zum Schluss gibt es noch einige Hinweise, wie Daten via dem WFS-Dienst gefiltert werden können (sowohl geografisch wie auch auf Attributebene).

Hinweis: die URL zum WFS Dienst finden man via Geoportal, welches auf dem OGD-Katalog auf den Geo-Datensätzen verlinkt ist.

In [57]:
import pandas as pd
import geopandas
import requests
import json
import xml.etree.ElementTree as ET
import folium

In [3]:
wfs_url = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Statistische_Quartiere"

## GetCapabilities

In [100]:
# GetCapabilities zeigt die möglichen Anfragen an einen WFS-Server
r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetCapabilities'
})
r.content

b'<WFS_Capabilities xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ogc="http://www.opengis.net/ogc" version="1.0.0" xmlns="http://www.opengis.net/wfs" updateSequence="0" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.0.0/WFS-capabilities.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:ows="http://www.opengis.net/ows">\n <Service>\n  <Name>WMS</Name>\n  <!-- Human-readable title for pick lists -->\n  <Title>QGIS mapserver</Title>\n  <!-- Narrative description providing additional information -->\n  <Abstract>A WMS service with QGIS mapserver</Abstract>\n  <KeywordList>\n   <Keyword>QGIS mapserver</Keyword>\n  </KeywordList>\n  <!-- Top-level web address of service or service provider.  See also OnlineResource\n\t\t     elements under <DCPType>. -->\n  <OnlineResource xlink:type="simple" xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="http://www.sourcepole.ch/"/>\n  <!-- Contact information 

In [53]:
# XML parsen und die Layer-Informationen extrahieren
root = ET.fromstring(r.content)
namespaces = {
    'wfs': 'http://www.opengis.net/wfs'
}
layers = {}
for feature_type in root.findall('wfs:FeatureTypeList/wfs:FeatureType', namespaces):
    layers[feature_type.find('wfs:Name', namespaces).text] = {
        'srs': feature_type.find('wfs:SRS', namespaces).text,
    }

layers

{'adm_statistische_quartiere_a': {'srs': 'EPSG:2056'},
 'adm_statistische_quartiere_b_p': {'srs': 'EPSG:2056'},
 'adm_statistische_quartiere_map': {'srs': 'EPSG:2056'},
 'adm_statistische_quartiere_v': {'srs': 'EPSG:2056'}}

Der Layer **adm_statistische_quartiere_map** ist der kartogratisch aufbereitete Layer. Die Details zu den einzelnen Layern sind auf dem OGD-Katalog beschrieben oder direkt auf geocat.ch.

Beispiel: https://www.geocat.ch/geonetwork/srv/ger/md.viewer#/full_view/fd1a94fe-4bd4-4a40-99af-8b859dfe82a7
![](geocat_layer_doc.png)

In [46]:
# Prüfen, welche Formate GetFeature bietet
formats = root.find('wfs:Capability/wfs:Request/wfs:GetFeature/wfs:ResultFormat', namespaces)
for child in formats:
    print(child.tag)

{http://www.opengis.net/wfs}GML2
{http://www.opengis.net/wfs}GML3
{http://www.opengis.net/wfs}GeoJSON


## Daten via GetFeature als GeoJSON laden

In [101]:
# Yay, GeoJSON!
# Daten als GeoJSON holen
layer = 'adm_statistische_quartiere_map'

r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': layer,
    'outputFormat': 'GeoJSON'
})
stat_quarter_geo = r.json()
stat_quarter_geo

{'type': 'FeatureCollection',
 'bbox': [8.44598942, 47.31845142, 8.62811572, 47.4357206],
 'features': [{'type': 'Feature',
   'id': 'adm_statistische_quartiere_map.1',
   'bbox': [8.55666, 47.353284, 8.590994, 47.37462],
   'geometry': {'type': 'Polygon',
    'coordinates': [[[8.556701, 47.364065],
      [8.556825, 47.364102],
      [8.556996, 47.364152],
      [8.557183, 47.364207],
      [8.557405, 47.364273],
      [8.557469, 47.364292],
      [8.557661, 47.364349],
      [8.557931, 47.364428],
      [8.557992, 47.364447],
      [8.558187, 47.364504],
      [8.558374, 47.36456],
      [8.558457, 47.364584],
      [8.558737, 47.364666],
      [8.558937, 47.364726],
      [8.559168, 47.364793],
      [8.559218, 47.364808],
      [8.55934, 47.364844],
      [8.55943, 47.364871],
      [8.559632, 47.36493],
      [8.559691, 47.364948],
      [8.55997, 47.36503],
      [8.56012, 47.365074],
      [8.560188, 47.365085],
      [8.560258, 47.365075],
      [8.560309, 47.36505],
      [8.56

Link to the GeoJSON: https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Statistische_Quartiere?service=WFS&version=1.0.0&request=GetFeature&outputFormat=GeoJSON&typename=adm_statistische_quartiere_map 

### Daten in GeoPandas als Tabelle

In [102]:
# load GeoJSON in geopandas
srs = layers[layer]['srs']
data = geopandas.GeoDataFrame.from_features(stat_quarter_geo, crs={'init': srs})
data

,geometry,kname,knr,objectid,objid,qname,qnr
0,"POLYGON ((8.556701 47.364065, 8.556825 47.3641...",Kreis 7,7,1,34,Hirslanden,73
1,"POLYGON ((8.556704999999999 47.364046, 8.55671...",Kreis 8,8,2,33,Weinegg,83
2,"POLYGON ((8.554233999999999 47.364557, 8.55425...",Kreis 8,8,3,32,Mühlebach,82
3,"POLYGON ((8.547946 47.365005, 8.548048 47.3648...",Kreis 8,8,4,31,Seefeld,81
4,"POLYGON ((8.533004999999999 47.373943, 8.53298...",Kreis 4,4,7,16,Werd,41
5,"POLYGON ((8.51937 47.374782, 8.519458999999999...",Kreis 3,3,8,15,Sihlfeld,34
6,"POLYGON ((8.501270999999999 47.379612, 8.50121...",Kreis 9,9,9,14,Albisrieden,91
7,"POLYGON ((8.583462000000001 47.388311, 8.58418...",Kreis 7,7,10,13,Hottingen,72
8,"POLYGON ((8.542973999999999 47.335432, 8.54285...",Kreis 2,2,5,30,Wollishofen,21
9,"POLYGON ((8.526883 47.407086, 8.526873 47.4071...",Kreis 6,6,18,22,Unterstrass,61


### Daten in Karte integrierten

In [103]:
# Basiskarte mit GeoJSON layer
m = folium.Map(location=[47.38, 8.53], zoom_start=13, tiles=None)
folium.raster_layers.WmsTileLayer(
    url='https://www.ogd.stadt-zuerich.ch/wms/geoportal/Uebersichtsplan_Stadt_Zuerich',
    layers='Uebersichtsplan_Stadt_Zuerich',
    name='Zürich - Basiskarte',
    fmt='image/png',
    overlay=False,
    control=False,
    autoZindex=False,
).add_to(m)
folium.features.GeoJson(stat_quarter_geo).add_to(m)
m

## Daten filtern

### Daten mit Attributen filtern

Um Attribute zu filtern, kann man sogenannte _OGC Filter_ erstellen, welche als XML formuliert werden müssen.
Diese können dann über den `Filter` Parameter dem `GetFeature` Request mitgegeben werden, so dass die Daten dann auf dem Server gefiltert werden.

In [85]:
# Filter für Kreis 7
filter_xml = """<?xml version="1.0"?>
<Filter xmlns="http://www.opengis.net/ogc" xmlns:gml="http://www.opengis.net/gml"> 
    <PropertyIsEqualTo>
        <PropertyName>knr</PropertyName>
        <Literal>7</Literal>
    </PropertyIsEqualTo>
</Filter>
"""

r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': layer,
    'outputFormat': 'GeoJSON',
    'Filter': filter_xml
})
filtered_geo = r.json()
filtered_data = geopandas.GeoDataFrame.from_features(filtered_geo, crs={'init': srs})
filtered_data

,geometry,kname,knr,objectid,objid,qname,qnr
0,"POLYGON ((8.556701 47.364065, 8.556825 47.3641...",Kreis 7,7,1,34,Hirslanden,73
1,"POLYGON ((8.583462000000001 47.388311, 8.58418...",Kreis 7,7,10,13,Hottingen,72
2,"POLYGON ((8.601855 47.371858, 8.601877 47.3717...",Kreis 7,7,14,25,Witikon,74
3,"POLYGON ((8.582151 47.387875, 8.58200199999999...",Kreis 7,7,24,11,Fluntern,71


### Daten geografisch filtern

Die Daten vom WFS lassen sich auch geografisch filtern. Wir können z.B. Polygone oder Bounding Boxes verwenden.

Im folgenden Beispiel holen wir uns alle Kirsch-Bäume aus dem Baumkataster, welche im Kreis 7 sind.

In [98]:
kreis7_geo_filter = """<?xml version="1.0"?>
<ogc:Filter xmlns:ogc="http://www.opengis.net/ogc">
    <ogc:And>
    <ogc:Intersects>
        <ogc:PropertyName>geometry</ogc:PropertyName>
        <gml:Box srsName="EPSG:2056">
            <gml:coordinates cs="," ts=" ">2684454.0423,1245344.4639 2687039.4349,1247714.1562</gml:coordinates>
        </gml:Box>
    </ogc:Intersects>
    <ogc:PropertyIsEqualTo>
        <ogc:PropertyName>baumgattunglat</ogc:PropertyName>
        <ogc:Literal>Prunus</ogc:Literal>
    </ogc:PropertyIsEqualTo>
    </ogc:And>
</ogc:Filter>
"""

baumkataster_wfs = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Baumkataster'
baum_layer = 'baumkataster_baumstandorte'
r = requests.get(baumkataster_wfs, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': baum_layer,
    'outputFormat': 'GeoJSON',
    'Filter': kreis7_geo_filter
})
tree_geo = r.json()
tree_data = geopandas.GeoDataFrame.from_features(tree_geo, crs={'init': srs})
tree_data

,baumartlat,baumgattunglat,baumnamedeu,baumnamelat,baumnummer,baumtyp,baumtyptext,genauigkeit,geometry,kategorie,objid,pflanzjahr,poi_id,quartier,status,strasse
0,None,Prunus,Zier-Kirsche,Prunus 'Umineko',FL-6708,5.0,"Höhe:<10m, Breite:<10m",Eingemessen,POINT (8.558152 47.372948),Strassenbaum,2818,2019.0,bm_2818,Fluntern,Strassenbaum,Attenhoferstr.
1,serrulata,Prunus,"Japanische Blüten-Kirsche, Japanische Nelken-K...",Prunus serrulata 'Kanzan',FL-6692,5.0,"Höhe:<10m, Breite:<10m",Eingemessen,POINT (8.558441999999999 47.373645),Strassenbaum,2819,2018.0,bm_2819,Fluntern,Strassenbaum,Spiegelhofstr. 39
2,serrulata,Prunus,"Japanische Blüten-Kirsche, Japanische Nelken-K...",Prunus serrulata 'Kanzan',FL-6693,5.0,"Höhe:<10m, Breite:<10m",Eingemessen,POINT (8.558661000000001 47.373849),Strassenbaum,2849,2018.0,bm_2849,Fluntern,Strassenbaum,Spiegelhofstr. 42
3,serrulata,Prunus,"Japanische Blüten-Kirsche, Japanische Nelken-K...",Prunus serrulata 'Kanzan',HO-4030,5.0,"Höhe:<10m, Breite:<10m",Eingemessen,POINT (8.561052999999999 47.373425),Strassenbaum,2850,2019.0,bm_2850,Hottingen,Strassenbaum,Spiegelhofstr. 62
4,serrulata,Prunus,"Japanische Blüten-Kirsche, Japanische Nelken-K...",Prunus serrulata 'Kanzan',HO-257,5.0,"Höhe:<10m, Breite:<10m",Eingemessen,POINT (8.558752999999999 47.372615),Strassenbaum,2878,NaN,bm_2878,Hottingen,Strassenbaum,Spiegelhofstr. 30
5,avium,Prunus,"Vogel-Kirsche, Süss-Kirsche (gefüllte)",Prunus avium 'Plena',FL-476,4.0,"Höhe:10-20m, Breite:<10m",Eingemessen,POINT (8.557956000000001 47.373021),Strassenbaum,2912,NaN,bm_2912,Fluntern,Strassenbaum,Attenhoferstr. 30
6,avium,Prunus,"Vogel-Kirsche, Süss-Kirsche (gefüllte)",Prunus avium 'Plena',FL-486,4.0,"Höhe:10-20m, Breite:<10m",Eingemessen,POINT (8.557729999999999 47.373196),Strassenbaum,2913,NaN,bm_2913,Fluntern,Strassenbaum,Attenhoferstr. 30
7,None,Prunus,Zier-Kirsche,Prunus 'Umineko',WI-2231,5.0,"Höhe:<10m, Breite:<10m",Bildschirmeingabe,POINT (8.587683999999999 47.360835),Strassenbaum,6101,2019.0,bm_6101,Witikon,Strassenbaum,Buchholzstr. 87
8,None,Prunus,Zier-Kirsche,Prunus 'Umineko',WI-2230,5.0,"Höhe:<10m, Breite:<10m",Bildschirmeingabe,POINT (8.587813000000001 47.360851),Strassenbaum,6102,2019.0,bm_6102,Witikon,Strassenbaum,Buchholzstr. 87
9,None,Prunus,Zier-Kirsche,Prunus 'Umineko',HI-2049,5.0,"Höhe:<10m, Breite:<10m",Bildschirmeingabe,POINT (8.578058 47.357862),Strassenbaum,6743,2019.0,bm_6743,Witikon,Strassenbaum,Eierbrechtstr. 47


In [99]:
tree_map = folium.Map(location=[47.38, 8.53], zoom_start=13, tiles=None)
folium.raster_layers.WmsTileLayer(
    url='https://www.ogd.stadt-zuerich.ch/wms/geoportal/Uebersichtsplan_Stadt_Zuerich',
    layers='Uebersichtsplan_Stadt_Zuerich',
    name='Zürich - Basiskarte',
    fmt='image/png',
    overlay=False,
    control=False,
    autoZindex=False,
).add_to(tree_map)
folium.features.GeoJson(tree_geo).add_to(tree_map)
tree_map